# Countlories

In [141]:
import os
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
from tensorflow.keras.applications import ResNet50
from keras.callbacks import ModelCheckpoint

### Preprocessing Data

Split data 80:20 karena dataset yang sedikit


In [116]:
#load source data
class_names = ['ayam_goreng', 'ayam_pop', 'daging_rendang', 'dendeng_batokok', 'gulai_ikan', 'gulai_tambusu', 'gulai_tunjang', 'lele_goreng', 'nasi', 'tahu_goreng', 'telur_dadar_goreng', 'telur_matasapi_goreng', 'telur_rebus_balado', 'tempe_goreng']
dataset = 'D:\\dataset - Copy'

#load data train dir 
root_dir = 'D:\\train'
train_dir = os.path.join(root_dir, 'training')
validation_dir = os.path.join(root_dir, 'validation')

def create_train_val_dirs(root_path, class_names):
    for food in class_names:
        train_food_dir = os.path.join(train_dir, food)
        validation_food_dir = os.path.join(validation_dir, food)
        if not os.path.exists(train_food_dir):
            os.makedirs(train_food_dir)
        if not os.path.exists(validation_food_dir):
            os.makedirs(validation_food_dir)

In [117]:
#cek
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

D:\train\training
D:\train\validation
D:\train\training\ayam_goreng
D:\train\training\ayam_pop
D:\train\training\daging_rendang
D:\train\training\dendeng_batokok
D:\train\training\gulai_ikan
D:\train\training\gulai_tambusu
D:\train\training\gulai_tunjang
D:\train\training\lele_goreng
D:\train\training\nasi
D:\train\training\tahu_goreng
D:\train\training\telur_dadar_goreng
D:\train\training\telur_matasapi_goreng
D:\train\training\telur_rebus_balado
D:\train\training\tempe_goreng
D:\train\validation\ayam_goreng
D:\train\validation\ayam_pop
D:\train\validation\daging_rendang
D:\train\validation\dendeng_batokok
D:\train\validation\gulai_ikan
D:\train\validation\gulai_tambusu
D:\train\validation\gulai_tunjang
D:\train\validation\lele_goreng
D:\train\validation\nasi
D:\train\validation\tahu_goreng
D:\train\validation\telur_dadar_goreng
D:\train\validation\telur_matasapi_goreng
D:\train\validation\telur_rebus_balado
D:\train\validation\tempe_goreng


In [118]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
    """
    Splits the data into train and validation sets.
  
    Args:
        SOURCE_DIR (string): directory path containing the images
        TRAINING_DIR (string): directory path to be used for training
        VALIDATION_DIR (string): directory path to be used for validation
        SPLIT_SIZE (float): proportion of the dataset to be used for training
    
    Returns:
        None
    """
  
    # Get the list of image files in the source directory
    source_files = os.listdir(SOURCE_DIR)
  
    # Shuffle the list of files randomly
    random.shuffle(source_files)
  
    # Calculate the split index based on the split size
    split_index = int(len(source_files) * SPLIT_SIZE)
  
    # Split the list of files into training and validation lists
    train_files = source_files[:split_index]
    validation_files = source_files[split_index:]
  
    # Copy the files to the respective directories
    for file in train_files:
        source_file = os.path.join(SOURCE_DIR, file)
        destination_file = os.path.join(TRAINING_DIR, file)
        shutil.copyfile(source_file, destination_file)
  
    for file in validation_files:
        source_file = os.path.join(SOURCE_DIR, file)
        destination_file = os.path.join(VALIDATION_DIR, file)
        shutil.copyfile(source_file, destination_file)

In [119]:
def split_data_combine(food):
    # Define paths
    FOOD_SOURCE_DIR = "D:\\dataset - Copy\\{}\\".format(food)
    
    TRAINING_DIR = "D:\\train\\training\\"
    VALIDATION_DIR = "D:\\train\\validation\\"
    
    TRAINING_FOOD_DIR = os.path.join(TRAINING_DIR, "{}\\".format(food))
    VALIDATION_FOOD_DIR = os.path.join(VALIDATION_DIR, "{}\\".format(food))
    
    if len(os.listdir(TRAINING_FOOD_DIR)) > 0:
        for file in os.scandir(TRAINING_FOOD_DIR):
            os.remove(file.path)
            
    if len(os.listdir(VALIDATION_FOOD_DIR)) > 0:
        for file in os.scandir(VALIDATION_FOOD_DIR):
            os.remove(file.path)
    
    split_size = 0.8
    
    split_data(FOOD_SOURCE_DIR, TRAINING_FOOD_DIR, VALIDATION_FOOD_DIR, split_size)


### Image Augmentation

In [120]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
    
    train_datagen = ImageDataGenerator(rescale=1./255.,
                                       rotation_range=30,       # Random rotation between -30 and 30 degrees
                                       width_shift_range=0.2,   # Randomly shift the width by 20%
                                       height_shift_range=0.2,  # Randomly shift the height by 20%
                                       shear_range=0.2,         # Randomly apply shear transformation
                                       zoom_range=0.2,          # Randomly zoom in or out by 20%
                                       horizontal_flip=True,    # Randomly flip the image horizontally
                                       vertical_flip=True,      # Randomly flip the image vertically
                                       brightness_range=(0.8, 1.2),  # Randomly adjust brightness between 0.8 and 1.2
                                       fill_mode='nearest')
    
    train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                        batch_size=32,
                                                        class_mode='categorical',
                                                        target_size=(224, 224))
    
    validation_datagen = ImageDataGenerator( rescale = 1./255.)

    validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                    batch_size=32,
                                                                    class_mode='categorical',
                                                                    target_size=(224, 224))
    return train_generator, validation_generator

In [121]:
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)
train_generator, test_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

Found 1111 images belonging to 14 classes.
Found 288 images belonging to 14 classes.
Found 1111 images belonging to 14 classes.
Found 288 images belonging to 14 classes.


## Model Train

In [103]:
!pip install tensorflow-hub

In [142]:
# Initialize ResNet50 model
ResNet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
ResNet50_model.trainable = False

# Print model summary
ResNet50_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_9[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

### Callback

In [124]:
class StopTrainingCallback(tf.keras.callbacks.Callback):
    def __init__(self, target_accuracy, patience=5):
        super(StopTrainingCallback, self).__init__()
        self.target_accuracy = target_accuracy
        self.patience = patience
        self.best_accuracy = 0
        self.wait = 0

    def on_epoch_end(self, epoch, logs=None):
        current_accuracy = logs['val_accuracy']
        if current_accuracy > self.best_accuracy:
            self.best_accuracy = current_accuracy
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience and current_accuracy < self.target_accuracy:
                print(f"\nAccuracy improvement below target accuracy of {self.target_accuracy} for {self.patience} epochs. Stopping training.")
                self.model.stop_training = True


callbacks = StopTrainingCallback(target_accuracy=0.92, patience=5)

callback_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="best_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=False,
    mode="max",
    verbose=1
)

In [125]:
def create_model():
    model = tf.keras.models.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(14, activation='softmax')
    ])
  
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['accuracy'])

    return model


In [107]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_4 (KerasLayer)  (None, 1001)              2664345   
                                                                 
 dense_32 (Dense)            (None, 512)               513024    
                                                                 
 dense_33 (Dense)            (None, 256)               131328    
                                                                 
 dense_34 (Dense)            (None, 128)               32896     
                                                                 
 batch_normalization_13 (Bat  (None, 128)              512       
 chNormalization)                                                
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                     

In [108]:
model = create_model()

history = model.fit(train_generator,
                    epochs=20,
                    verbose=1,
                    validation_data=validation_generator,
                    callbacks=[callbacks, callback_model_checkpoint])


Epoch 1/20
35/35 [==============================] - ETA: 0s - loss: 1.8354 - accuracy: 0.4329
Epoch 1: val_accuracy improved from -inf to 0.61458, saving model to best_model.h5
35/35 [==============================] - 39s 1s/step - loss: 1.8354 - accuracy: 0.4329 - val_loss: 1.3127 - val_accuracy: 0.6146
Epoch 2/20
35/35 [==============================] - ETA: 0s - loss: 0.8782 - accuracy: 0.7057
Epoch 2: val_accuracy improved from 0.61458 to 0.76042, saving model to best_model.h5
35/35 [==============================] - 31s 892ms/step - loss: 0.8782 - accuracy: 0.7057 - val_loss: 0.6933 - val_accuracy: 0.7604
Epoch 3/20
35/35 [==============================] - ETA: 0s - loss: 0.6823 - accuracy: 0.7768
Epoch 3: val_accuracy improved from 0.76042 to 0.80208, saving model to best_model.h5
35/35 [==============================] - 32s 893ms/step - loss: 0.6823 - accuracy: 0.7768 - val_loss: 0.6085 - val_accuracy: 0.8021
Epoch 4/20
35/35 [==============================] - ETA: 0s - loss: 0.

In [110]:
model_results_1 = model.evaluate(validation_generator)

9/9 [==============================] - 4s 474ms/step - loss: 0.4814 - accuracy: 0.8576


## Test


In [126]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img, img_to_array
from PIL import Image

image_path = input("Enter the image path: ")

# Display the selected image path
print("Selected image path:", image_path)

# Load and preprocess the image
img = load_img(image_path, target_size=(224, 224))
x = img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

# Predict the image class
classes = model.predict(images, batch_size=1)
label_name = list(train_generator.class_indices.keys())[np.argmax(classes)]
print(f"Predicted probabilities: {classes}")
print(f"Predicted category: {np.argmax(classes)}")
print(f"Predicted label: {label_name}")

Enter the image path: D:\images.jpg
Selected image path: D:\images.jpg
1/1 [==============================] - 0s 46ms/step
Predicted probabilities: [[3.2810038e-03 2.7228376e-02 2.1038542e-04 1.1234645e-04 4.0240525e-04
  1.6489939e-03 2.4157928e-02 9.2043556e-05 2.7563199e-04 8.2696301e-01
  7.7674200e-04 1.4625726e-02 1.0052536e-03 9.9220201e-02]]
Predicted category: 9
Predicted label: tahu_goreng


## Save
.h5

In [113]:
import os
from tensorflow.keras.models import save_model

# Define the file path for the H5 file
model_path = 'D:\\model.h5'

# Check if the file exists and delete it if it does
if os.path.isfile(model_path):
 os.remove(model_path)

# Save the model
save_model(model, model_path)

.pb

In [129]:
# Save the model in the .pb format
saved_model_dir = "saved"
tf.saved_model.save(model, saved_model_dir)


INFO:tensorflow:Assets written to: saved\assets


INFO:tensorflow:Assets written to: saved\assets
